In [229]:
#==========IMPORTS==========

#----------LIBRARIES----------
from ultralytics import YOLO
import cv2
import numpy as np

#----------GITS----------
from Libraries.sort import Sort

In [230]:
#==========MODELS LOAD==========

#----------MODEL AND SORT LOAD----------
Model_Plates = YOLO('YOLOv8/best.pt')
Model_Number = YOLO('Number/best.pt')

#----------DICTIONARIES----------
Current_ID = {}

#----------SORT----------
Tracker = Sort()

In [231]:
#==========MODELS SYSTEMS==========

#----------NUMBERS MODEL----------
def Plates_Number(Model_Number):
    if 0 <= Model_Number <= 9:
        return chr(ord('0') + Model_Number)
    elif 10 <= Model_Number <= 35:
        return chr(ord('A') + (Model_Number - 10))
    else:
        return ''
    
#----------PLATES MODEL ACTIVATION----------
def Plates_Model_Activation(Frame):
    Model_Data = Model_Plates(Frame)
    Detection = []

    for Result in Model_Data[0].boxes:
        Class = int(Result.cls[0])
        if Result.xyxy[0].shape[0] >= 4:
            x1, y1, x2, y2 = Result.xyxy[0][:4].tolist()
            Score = Result.conf[0].item()

            if Class == 1:
                Detection.append([x1, y1, x2, y2, Score])
                
    return Detection

#----------NUMBERS MODEL ACTIVATION----------
def Numbers_Model_Activation(Plate_Area, x1, y1, x2, y2):
    Model_Data = Model_Number(Plate_Area)
    Detection = []

    for Result in Model_Data[0].boxes:
        Class = int(Result.cls[0])
        Character = Plates_Number(Class)
        Char_x1, Char_y1, Char_x2, Char_y2 = Result.xyxy[0][:4].tolist()

        if (Char_x1 >= 0 and Char_x2 <= (x2 - x1)) and (Char_y1 >= 0 and Char_y2 <= (y2 - y1)):
            Detection.append((Character, Char_x1))
    
    return Detection

In [232]:
#==========TRACKING SYSTEMS==========

#----------PLATES TRACKER----------
def Plates_Tracker(Positions):
    Positions.sort(key=lambda x: x[1])
    Plate_Text = ''.join([c[0] for c in Positions])
    
    return Plate_Text

#----------PLATES PRINT ID----------
def Plates_Print(Current_ID, Plate_ID, Frame, x1, x2, y1, y2):
    Plate_Number = Current_ID[Plate_ID]
    Text = f"ID: {int(Plate_ID)} Plate Number: {Plate_Number}"
    cv2.rectangle(Frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.putText(Frame, Text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

In [233]:
#==========PLATES SYSTEMS==========

#----------PLATES PROCESS----------
def Plates_Processing_Info(Frame, Current_ID, Plate_ID, x1, x2, y1, y2):
    Plate_Area = Frame[y1:y2, x1:x2]
    Plate_Image = Plate_Area
    
    Character_Position = Numbers_Model_Activation(Plate_Area, x1, y1, x2, y2)
    Current_ID[Plate_ID] = Plates_Tracker(Character_Position)

    return Plate_Image, Current_ID[Plate_ID]

#----------PLATES BOUNDING BOXES PROCESS----------
def Plates_Bounding_Boxes(Frame, Plate_ID, Plate_Image, x1, x2, y1, y2):
    if Plate_ID not in Current_ID:
        Plate_Image, Current_ID[Plate_ID] = Plates_Processing_Info(Frame, Current_ID, Plate_ID, x1, x2, y1, y2)

    Plates_Print(Current_ID, Plate_ID, Frame, x1, x2, y1, y2)

    return Plate_Image, Current_ID[Plate_ID]

#----------TRACKING PROCESS----------
def Tracking_Processing(Frame, NP_Detection, Bottom_Right_Area, Plate_Image):
    Objects_Tracked = Tracker.update(NP_Detection)
    Tracked_IDs = []
    
    for Object in Objects_Tracked:
        x1, y1, x2, y2, Plate_ID = Object
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        Tracked_IDs.append(Plate_ID)

        if (x1 >= Bottom_Right_Area[0] and y1 >= Bottom_Right_Area[1]):
            Plate_Image, Current_ID[Plate_ID] = Plates_Bounding_Boxes(Frame, Plate_ID, Plate_Image, x1, x2, y1, y2)

    if not any(Plate_ID in Tracked_IDs for Plate_ID in Current_ID.keys()):
        Plate_Image = None

    return Plate_Image

In [234]:
#==========SCREEN SYSTEMS==========

#----------SCREEN SIZE----------
def Screen_Size(Frame):
    Frame_Resized = cv2.resize(Frame, (900, 500))
    cv2.imshow('COIMATRON', Frame_Resized)

    return Frame_Resized

#----------SCREEN RESIZE----------
def Resize_Screen(Frame):
    Frame_Width = int(Frame.get(cv2.CAP_PROP_FRAME_WIDTH))
    Frame_Height = int(Frame.get(cv2.CAP_PROP_FRAME_HEIGHT))

    Resized_Frame = (Frame_Width // 2, Frame_Height // 2, Frame_Width, Frame_Height)

    return Resized_Frame, Frame_Width

#----------SCREEN OPTIONS----------
def Screen_Options(Pause, Frame):
    Key = cv2.waitKey(1) & 0xFF
    if Key == ord('z'):
        return True, False
    
    elif Key == ord('x'):
        return False, True
    
    if Pause:
        cv2.imshow('COIMATRON', Frame)
        while True:
            Key = cv2.waitKey(1) & 0xFF
            if Key == ord('z'):
                return False, False
            
            elif Key == ord('x'):
                return False, True
    
    return False, False

#----------ROI FRAME----------
def ROI_Frame(Frame, Bottom_Right_Area):
    ROI_Area = Frame[Bottom_Right_Area[1]:Bottom_Right_Area[3], Bottom_Right_Area[0]:Bottom_Right_Area[2]]
    return ROI_Area

#----------PLATE RESIZE----------
def Plate_Image_Resize(Frame, Plate_Image, Frame_Width):
    if Plate_Image is not None:
        Resized_PLate_Image = cv2.resize(Plate_Image, (200, 100))
        Frame[0:100, Frame_Width - 200:Frame_Width] = Resized_PLate_Image
    return Frame

In [235]:
#==========OVERALL SYSTEM==========

#----------SYSTEM IMPLEMENTATION----------
def Artificial_Inteligence_Implementation(Window, Bottom_Right_Area, Plate_Image):
    _, Frame = Window.read()
    ROI_Frame(Frame, Bottom_Right_Area)
    Detections = Plates_Model_Activation(Frame)
    NP_Detections = np.array(Detections)

    if NP_Detections.shape[0] > 0:
        Plate_Image = Tracking_Processing(Frame, NP_Detections, Bottom_Right_Area, Plate_Image)

    return Plate_Image, Frame

In [236]:
#==========MAIN CODE==========

#----------CAMERA----------
Window = cv2.VideoCapture('IMG_1265.MOV')

#----------PAUSE AND CLOSE STATE----------
Pauser = False
Closer = False

#----------WINDOW PREPROCESSING
Bottom_Right_Area, Frame_Width = Resize_Screen(Window)
Plate_Image = None

#----------WHILE LOOP----------
while True:
    if not Pauser:
        Plate_Image, Frame = Artificial_Inteligence_Implementation(Window, Bottom_Right_Area, Plate_Image)
        Frame = Plate_Image_Resize(Frame, Plate_Image, Frame_Width)
        Resized_Frame = Screen_Size(Frame)
    
    Pauser, Closer = Screen_Options(Pauser, Resized_Frame)
    if Closer:
        break

Window.release()
cv2.destroyAllWindows()


0: 384x640 4 Autos, 2 Placas, 4.7ms
Speed: 2.0ms preprocess, 4.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 5.0ms
Speed: 2.0ms preprocess, 5.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 6.5ms
Speed: 2.7ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 5.9ms
Speed: 3.2ms preprocess, 5.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 5.9ms
Speed: 1.6ms preprocess, 5.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 5.9ms
Speed: 2.0ms preprocess, 5.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 5.5ms
Speed: 1.7ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Autos, 2 Placas, 7.1ms
Speed: 2.5ms preprocess, 7.1ms i